# Laboratorio no. 5 Threat Hunting
Marco Jurado 20308

In [22]:
#pip install python-whois

In [23]:
import pandas as pd
import json
import whois

## Parte 1

Abrimos large_eve.json y luego verificamos tener los elementos correctos

In [24]:
data = []

with open('large_eve.json', 'r') as file:
    for line in file:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print("ERROR al decodificar JSON en la línea:", e)

print("---> Total de registros cargados: ",len(data))

---> Total de registros cargados:  746909


In [25]:
df = pd.DataFrame(data)
df

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
0,2017-07-22T17:33:15.784100-0500,2.835707e+14,18519.0,alert,130.0,192.168.203.200,2328.0,192.181.145.109,445.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-07-22T17:33:16.571273-0500,1.519048e+15,21531.0,ssh,140.0,192.168.204.60,62990.0,192.168.26.254,22.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-07-22T17:33:17.561063-0500,9.632021e+14,28731.0,http,140.0,192.168.204.59,38011.0,192.168.24.201,80.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-07-22T17:33:17.595416-0500,9.632021e+14,28994.0,fileinfo,140.0,192.168.24.201,80.0,192.168.204.59,38011.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746904,2017-07-22T19:47:49.301548-0500,1.970284e+15,NaN,flow,NaN,192.168.203.200,2007.0,192.183.45.157,445.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746905,2017-07-22T19:47:49.301548-0500,2.251792e+15,NaN,flow,NaN,192.168.198.57,52498.0,192.168.21.141,22.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746906,2017-07-22T19:47:49.301548-0500,8.444188e+14,NaN,flow,NaN,192.168.201.72,57805.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN
746907,2017-07-22T19:47:49.301548-0500,2.814688e+14,NaN,flow,NaN,192.168.201.72,43004.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN


Una vez abierto correctamente y generado nuestro set de datos con pandas. Vamos a proceder a abrir el archivo con los DNS y filtramos estos para verificar si tenemos exito al obtener los elementos.

In [26]:
recordsDNS = [record for record in data if record.get('event_type') == 'dns']
print(" ---> Total de registros DNS: ",len(recordsDNS))

 ---> Total de registros DNS:  15749


Ahora que tenemos el data frame de DNS vamos a verificar la forma del mismo data frame.

In [27]:
dfDNS = pd.json_normalize(recordsDNS)
print(" ---> Forma del DataFrame de DNS: ", dfDNS.shape)
dfDNS.head()

 ---> Forma del DataFrame de DNS:  (15749, 18)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


In [28]:
dfDNSrecords = dfDNS[dfDNS['dns.rrtype'] == 'A']
print(dfDNSrecords.sample(2))

                             timestamp           flow_id  pcap_cnt event_type  \
11415  2017-07-22T19:29:18.762270-0500  1964791037862302   3354273        dns   
4557   2017-07-22T18:29:17.165146-0500   189878386918682    504116        dns   

       vlan           src_ip  src_port        dest_ip  dest_port proto  \
11415   150  192.168.205.188     52775  192.168.207.4         53   UDP   
4557    150  192.168.205.188     49984  192.168.207.4         53   UDP   

      dns.type  dns.id             dns.rrname dns.rrtype  dns.tx_id dns.rcode  \
11415    query   15902   mirrors.xmission.com          A        0.0       NaN   
4557     query    7130  mirror.its.uidaho.edu          A        0.0       NaN   

       dns.ttl dns.rdata  
11415      NaN       NaN  
4557       NaN       NaN  


Mostramos dos elementos cualquiera

In [29]:
json.dumps(recordsDNS[0], indent=2)

'{\n  "timestamp": "2017-07-22T17:33:16.661646-0500",\n  "flow_id": 1327836194150542,\n  "pcap_cnt": 22269,\n  "event_type": "dns",\n  "vlan": 110,\n  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",\n  "src_port": 59680,\n  "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",\n  "dest_port": 53,\n  "proto": "UDP",\n  "dns": {\n    "type": "query",\n    "id": 15529,\n    "rrname": "api.wunderground.com",\n    "rrtype": "A",\n    "tx_id": 0\n  }\n}'

In [30]:
json.dumps(recordsDNS[1], indent=2)

'{\n  "timestamp": "2017-07-22T17:33:24.990320-0500",\n  "flow_id": 2022925111925872,\n  "pcap_cnt": 54352,\n  "event_type": "dns",\n  "vlan": 110,\n  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",\n  "src_port": 38051,\n  "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",\n  "dest_port": 53,\n  "proto": "UDP",\n  "dns": {\n    "type": "query",\n    "id": 58278,\n    "rrname": "stork79.dropbox.com",\n    "rrtype": "A",\n    "tx_id": 0\n  }\n}'

Mostramos la forma del detaframe de DNS

In [31]:
print(dfDNS.shape)

(15749, 18)


filtrar los registros DNS para aquellos registros tipo A

In [32]:
print("Total de registros tipo 'A': ", len(dfDNSrecords))

Total de registros tipo 'A':  2849


Filtrar los dominios únicos

In [33]:
dominiosUnicos = dfDNSrecords['dns.rrname'].unique()
print("Total de dominios únicos tipo 'A': ", len(dominiosUnicos))


Total de dominios únicos tipo 'A':  177


Función que extraiga el TLD de un dominio completo

In [34]:
from urllib.parse import urlparse

def get_tld(domain):
    if domain:
        if not domain.startswith(('http://', 'https://')):
            domain = 'http://' + domain
        tld = urlparse(domain).hostname.split('.')[-2:]
        return '.'.join(tld) if len(tld) == 2 else domain
    return ""

print(get_tld("api.wunderground.com"))
print(get_tld("safebrowsing.clients.google.com.home"))


wunderground.com
com.home


In [35]:
dfUnicosElementosA = pd.DataFrame(dominiosUnicos, columns=['domain'])
dfUnicosElementosA['domain_tld'] = dfUnicosElementosA['domain'].apply(get_tld)
dfUnicosElementosA = dfUnicosElementosA.drop(columns=['domain'])
dfUnicosElementosA

,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,com.home
4,mozilla.com
...,...
172,com.localdomain
173,microsoft.com
174,verisign.com
175,22.201


## Parte 2

Primero vamos a realizar la importación del clasificador adjuntado en el proyecto.

In [36]:
from clasificador import *

Iniciamos el proceso de filtrar aquellos considerados como DGA (valor 1 ).

In [37]:
dfDGAvalor1 = clasificacion(dfUnicosElementosA)
df_dga = dfDGAvalor1[dfDGAvalor1['isDGA'] == 1]
dfDGAunicos = df_dga.drop_duplicates(subset=['domain_tld'])
print(" ---> Total de dominios únicos considerados DGA: ", len(dfDGAunicos))
dfDGAunicos

 ---> Total de dominios únicos considerados DGA:  39


C:\Users\marco\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,domain_tld,isDGA
1,dropbox.com,1
2,aoltw.net,1
5,metasploit.com,1
18,22.110phpmyadmin,1
24,110phpmyadmin.localdomain,1
25,windows.com,1
27,ntkrnlpa.info,1
34,sql-ledger.org,1
35,backtrack-linux.org,1
43,hackerlabs.vpn,1


## Parte 3

Creación de función con el promt de chatGPT:

"Quiero escribir una función que sea capaz de tomar un listado de elementos de tipo dominios considerados como sospechosos. Vas a utilizar una lista de un millón de TLD y para esta función debes de devolver el valor '0' si el TLD que evaluarás se encuentra en el listado y '1' si no está."

In [38]:
top_1m_file_path = 'top-1m.csv'
with open(top_1m_file_path, 'r') as file:
    top_1m_tlds = set(line.strip().split(',')[1] for line in file)

def check_tld_in_list(tld):
    """
    Devuelve 0 si el TLD se encuentra en la lista de un millón de TLDs,
    y 1 si no está.
    """
    return 0 if tld in top_1m_tlds else 1

In [39]:
dfDGAunicos = dfDGAunicos.copy()
dfDGAunicos.loc[:, 'in_known_list'] = dfDGAunicos['domain_tld'].apply(check_tld_in_list)
df_suspected_domains = dfDGAunicos[dfDGAunicos['in_known_list'] == 1]
df_suspected_domains = df_suspected_domains.drop_duplicates(subset=['domain_tld'])
print(" ---> Total de dominios sospechosos únicos: ", len(df_suspected_domains))


 ---> Total de dominios sospechosos únicos:  17


Creamos una función basada en el TLD para la fecha de creación

In [40]:
def obtener_fecha_creacion(tld):
    try:
        dominio_whois = whois.whois(tld)
        return dominio_whois.creation_date
    except Exception as e:
        return f"Error al obtener la fecha: {e}"

In [41]:
df_suspected_domains['fecha_creacion'] = df_suspected_domains['domain_tld'].apply(obtener_fecha_creacion)

In [42]:
df_suspected_domains[['domain_tld', 'fecha_creacion']]

,domain_tld,fecha_creacion
2,aoltw.net,2000-01-10 04:09:19
18,22.110phpmyadmin,None
24,110phpmyadmin.localdomain,None
27,ntkrnlpa.info,2023-06-27 08:18:55
34,sql-ledger.org,2000-09-08 16:25:09
35,backtrack-linux.org,2009-04-29 00:26:42
43,hackerlabs.vpn,None
64,bigflickrfeed.com,"Error al obtener la fecha: No match for ""BIGFL..."
66,http://wpad,None
82,malwarecity.com,"[2008-02-06 15:42:35, 2008-02-06 00:00:00]"


Ahora vamos a ver los elementos que tenemos en la tabla y al ojo crítico vamos a determinar cuales de estos se ven sospechosos según criterios que hemos visto en clase y en los laboratorios pasados.

- ejfodfmfxlkgifuf.xyz
- vtlfccmfxlkgifuf.com
- cakephp.org
- 110phpmyadmin.localdomain
- 22.110phpmyadmin
- malwarecity.com